In [142]:
%load_ext autoreload
%autoreload 2

import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [143]:
!conda info --envs

# conda environments:
#
base                     /Users/diat.lov/miniconda3
audio                    /Users/diat.lov/miniconda3/envs/audio
chatbot               *  /Users/diat.lov/miniconda3/envs/chatbot
cv                       /Users/diat.lov/miniconda3/envs/cv
gymlab                   /Users/diat.lov/miniconda3/envs/gymlab
hyperstyle               /Users/diat.lov/miniconda3/envs/hyperstyle
pythonProject1           /Users/diat.lov/miniconda3/envs/pythonProject1
rl                       /Users/diat.lov/miniconda3/envs/rl



In [268]:
from data_preprocessing.tg_json_reader import DataRetriever
from data_preprocessing.conversation_builder import create_conversations

In [240]:
path = "/Users/diat.lov/Downloads/Telegram Desktop/DataExport_2021-11-05/result.json"
data_retriever = DataRetriever(path)
dictinoary = data_retriever.get_message_history_by_name("Катя")

In [241]:
df = data_retriever.get_pandas_df(dictinoary)

In [185]:
df.head()

,id,type,date,from,from_id,text,contact_information,file,thumbnail,media_type,...,discard_reason,location_information,forwarded_from,edited,via_bot,performer,title,live_location_period_seconds,poll,message_id
0,1,message,2017-06-05T08:52:50,Daria Diatlova,user336968719,D,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,message,2017-06-05T08:55:30,Катя,user109937743,,"{'first_name': 'Светлана Анатольевна(айф)', 'l...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11,message,2017-06-06T03:16:59,Daria Diatlova,user336968719,Нормальная рубашка😅😅,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,13,message,2017-06-06T03:24:19,Катя,user109937743,Пхахахаххахахаха,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,14,message,2017-06-06T03:24:37,Катя,user109937743,Ты когда домой?,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [242]:
df = data_retriever.tg_specific_preprocessing(df)

In [235]:
df.head(5)

,id,date,time,from,text
0,1,2017-06-05,08:52:50,Daria,D
1,11,2017-06-06,03:16:59,Daria,Нормальная рубашка😅😅
2,13,2017-06-06,03:24:19,Kate,Пхахахаххахахаха
3,14,2017-06-06,03:24:37,Kate,Ты когда домой?
4,15,2017-06-06,03:30:55,Daria,В 8


In [228]:
def create_conversations(df):
    conversations = []
    idx = 0
    min_dialoge_len = 5
    max_dialoge_len = 100
    while idx < df.shape[0]:
        target_hour = df.time.loc[idx].hour
        message_block = [df.loc[idx]["from"] + " " + df.loc[idx]["text"]]
        for i in range(idx + 1, df.shape[0]):
            if df.time.loc[i].hour == target_hour:
                message_block.extend([df.loc[i]["from"] + " " + df.loc[i]["text"]])
            else:
                target_hour = df.time.loc[i].hour
                idx += i
                if min_dialoge_len <= len(message_block) <= max_dialoge_len:
                    conversations.append(message_block)
                break 
    return conversations

In [229]:
small_df = df.loc[0:200]

In [230]:
small_conversations = create_conversations(small_df)

In [234]:
small_conversations[2]

['Kate Отлично',
 'Daria У тебя дома велик?',
 'Kate Да',
 'Daria Скажем, что ты спортом занималась',
 'Kate Хорошо',
 'Kate Я напишу',
 'Daria Все, Коля поехал на работу, я сказала, что ты написала, что бегала и уже домой бежишь',
 'Daria Он спросил, что мы будем делать, я сказала, что Коли отвезёт нас куда-нибудь в город',
 'Daria Коди*',
 'Daria Ой, я себе в личку кидала']

In [269]:
all_conversations = create_conversations(df)

In [283]:
len(all_conversations)

4741

In [286]:
all_conversations[10]

['Kate Ты работаешь ещё?',
 'Kate Ты работаешь ещё?',
 'Daria Нет, сижу просто',
 'Kate Коди попросить тебя забрать?',
 'Kate Напиши что тебе сделать',
 'Kate Хорошо',
 'Kate Напиши как можешь',
 'Kate Поедешь',
 'Kate Чай поставлю',
 'Daria Оки']